# Image Slice inferencing
Testing image slice inferenceing for microservice setup

# 1) Imports and mount

In [1]:
%load_ext autoreload
%autoreload 2

# This sets up the appropriate logging and path configs
from notebook_setup import * 

In [2]:
import torch
from torchsummary import summary
import matplotlib.pyplot as plt

from monai.losses import DiceLoss
from src.features.build_features import train_transform, val_transform, test_transform

from src.visualization.visualize import view_slice

from monai.data import Dataset, DataLoader
from src.enums import DataDict

from src.settings.config import get_app_settings
from src.utils import load_yaml

## 2) Import Settings

In [3]:
settings = get_app_settings()
model_config = load_yaml("model_configs.yaml")

## 3) Inference



### 3.1) Infernce on volume nii

In [4]:
# Get sample volume 
from src.data.make_dataset import get_test_raw_paths


raw_test_paths = get_test_raw_paths()

In [5]:
from copy import copy
sample = copy(raw_test_paths[0])
import nibabel as nib
# t1_sample = nib.load(sample[DataDict.ImageT1])
# flair_sample = nib.load(sample[DataDict.ImageFlair])
# label_sample = nib.load(sample[DataDict.Label])
t1_sample = nib.load(sample[DataDict.ImageT1]).get_fdata()
flair_sample = nib.load(sample[DataDict.ImageFlair]).get_fdata()
label_sample = nib.load(sample[DataDict.Label]).get_fdata()

sample[DataDict.ImageT1] = t1_sample
sample[DataDict.ImageFlair] = flair_sample
sample[DataDict.Label] = label_sample

In [6]:
# sample

In [7]:
sample_data_paths = [sample]
# sample_dadta_paths

In [8]:
from src.features.build_features import inference_transform
test_dataset = Dataset(sample_data_paths, inference_transform)

In [9]:

subject_data = test_dataset.__getitem__(0)
subject_data

RuntimeError: applying transform <monai.transforms.compose.Compose object at 0x00000229E4FF76D0>

In [ ]:
from src.models.predict_model import ImagePredictor
from src.models.train_model import model

pred_network = model
checkpoint = torch.load('models/single_slice_t1_flair_v1.pt', map_location=torch.device('cpu'))
pred_network.load_state_dict(checkpoint['model_state_dict'])
img_predictor = ImagePredictor(pred_network, test_dataset)
test_predictions = img_predictor.predict_handler()
# print('DONE')

RuntimeError: applying transform <monai.transforms.compose.Compose object at 0x000002C6A6840650>

In [ ]:
from monai.inferers import inferer
from monai.transforms import Compose
model.eval()
device = torch.device("cpu")
transform = Compose([
    ToTensor(), 
    LoadImage(image_only=True)
    ])
data = transform(sample_data_paths).to(device)
with torch.no_grad():
    pred = inferer(inputs=data, network=model)